# Imports

In [1]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.CodeMirror{font-family: "Courier New";font-size: 12pt;}</style>'))

In [2]:
import findspark
findspark.init()

In [3]:
import re
import os
import pandas
pandas.set_option('display.max_rows', None)

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import colors
from matplotlib import rcParams
rcParams['font.sans-serif'] =  'Courier New'
rcParams['font.family'] = 'Courier New'
rcParams['font.size'] = '12'
%matplotlib inline

from IPython.display import display,HTML
import threading
import collections

from IPython.display import display
from ipywidgets import IntProgress,Layout
import time
import threading
import gzip
import pyspark
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructType, StructField, DateType,
    TimestampType, StringType, LongType, IntegerType, DoubleType,FloatType)
from pyspark.sql.functions import to_date, floor
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit
import datetime
import time
from pyspark.storagelevel import StorageLevel
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.ml import Pipeline
import pandas
import numpy


import re
import math
from functools import reduce
import json


from pyspark.sql.types import *
from pyspark.sql import functions as F
from datetime import date

# start analysis cluster and run

In [4]:
executors_per_node = 4
nodes=1
cores_per_executor=4
task_per_core=2


cache_size=100
total_size=340000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
conf = (SparkConf()
    .set("spark.default.parallelism", "{:d}".format(nodes*executors_per_node))
    .set('spark.executor.instances', '{:d}'.format(nodes*executors_per_node))
    .set('spark.files.maxPartitionBytes', '256m')
    .set('spark.app.name', 'pyspark_final')
    .set('spark.rdd.compress', 'False')
    .set('spark.serializer','org.apache.spark.serializer.KryoSerializer')    
    .set('spark.executor.cores','{:d}'.format(cores_per_executor))
    .set('spark.sql.shuffle.partitions','{:d}'.format(nodes*executors_per_node))
    .set('spark.executor.memory', '{:d}m'.format(int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
    .set('spark.task.cpus','1')
    .set('spark.driver.memory','128g')
    .set('spark.sql.inMemoryColumnarStorage.compressed','False')
    .set('spark.sql.inMemoryColumnarStorage.batchSize','100000')
#    .set('spark.memory.storageFraction','0.8')
#    .set('spark.memory.fraction','0.7')
        
    .set('spark.sql.execution.arrow.fallback.enabled','True')
    .set('spark.sql.execution.arrow.enabled','True')
    .set('spark.sql.execution.arrow.maxRecordsPerBatch','100000')
    .set("spark.sql.repl.eagerEval.enabled", True)
        
    .set('spark.memory.offHeap.enabled','True')
    .set('spark.memory.offHeap.size','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
    .set('spark.executor.memoryOverhead','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))+3000))
    .set('spark.sql.join.preferSortMergeJoin','False')
#    .set('spark.executor.extraJavaOptions',
#          '-XX:+UseG1GC -XX:+PrintFlagsFinal -XX:+PrintReferenceGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps -XX:InitiatingHeapOccupancyPercent=35')
    .set('spark.executor.extraJavaOptions',
          '-XX:+UseParallelGC -XX:+UseParallelOldGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps')
    .set('spark.dynamicAllocation.enabled', 'False')
    .set('spark.shuffle.service.enabled', 'False')
    .set('spark.dynamicAllocation.shuffleTracking.enabled', 'False')
    .set("spark.sql.legacy.timeParserPolicy","LEGACY")

        
#    .set("spark.sql.session.timeZone", "PST")
     .set("spark.sql.session.timeZone", "Etc/GMT-8")
#     .set("spark.driver.extraJavaOptions","-Duser.timezone=UTC+8")
    
       )

sc = SparkContext(conf=conf,master='yarn')
sc.setLogLevel("ERROR")
spark = SQLContext(sc)
time.sleep(10)


executor per node: 4
parallelism: 32
memory: 58376m
offheap:25600m


# Sparklog

In [5]:
%run sparklog.ipynb

In [6]:
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [7]:
appals=App_Log_Analysis("/user/root/spark_logs/application_1645145794788_0014",None)

In [8]:
appals.load_data()

load data  /user/root/spark_logs/application_1645145794788_0014


# Basic Info

In [ ]:
appals.get_basic_state()

In [ ]:
appals.get_app_info()

# Shuffle State

In [ ]:
shuffle_df, dfx=appals.get_shuffle_stat()

# TraceView

In [ ]:
#  The command generate the traceview as json format. It can be open by traceviewer in https://chromium.googlesource.com/catapult.
#  1. clone catapult
#  2. copy/generate the traceview json file to a folder like /home/xxx/trace_result
#  3. cd catapult/bin/
#  4. python2.7 ./run_dev_server --no-install-hooks -d /home/xxx/trace_result -p1088
#  5. open like in browser: http://127.0.0.1:1088/tracing_examples/trace_viewer.html#/tracing/test_data/application_1647347981137_0221_traceview.json

#  if you need to open the traceview from hostip instead of 127.0.0.1, hack catapult :-)

In [ ]:
appals.generate_trace_view("/home/xxx/" + appals.appid + "_traceview")

# hot queries

In [ ]:
# querys which has biggest gap compared to vanilla
for q in []:
    appals.get_query_plan(queryid=q)

# ColumnarToRow Queries

In [1]:
# change the node name to ColumnarToRow if native columnarToRow is disabled

In [ ]:
c2r=appals.get_metrics_by_node("ArrowColumnarToRow")
c2rstagedf=c2r.where("`number of output rows` > 1000000").where("ElapsedTime>100").select("real_queryid","Stage ID").distinct().orderBy("Stage ID").collect()
c2rstage=[l['Stage ID'] for l in c2rstagedf]

In [ ]:
len(c2rstage)

In [ ]:
for s in c2rstage:
    v=appals.get_query_plan(stageid=s,)

In [ ]:
c2rquery=set([l['real_queryid'] for l in c2rstagedf])

In [ ]:
for q in c2rquery:
    appals.get_query_plan(queryid=q)

## physical plans

In [ ]:
#for q in c2rquery:
#    ps=appals.get_physical_plan(queryid=q)

# RowToColumnar

In [ ]:
# change the node name to RowToArrowColumnar if native RowToColumnar is disabled

In [ ]:
r2c=appals.get_metrics_by_node("ArrowRowToColumnarExec")
r2cstage=r2c.where("`number of input rows` > 1000000").where("ElapsedTime>100").select("Stage ID").distinct().orderBy("Stage ID").collect()
r2cstage=[l['Stage ID'] for l in r2cstage]

In [ ]:
len(r2cstage)

In [ ]:
for s in [l for l in r2cstage if l not in c2rstage]:
    v=appals.get_query_plan(stageid=s,)

# physical plans

In [ ]:
p=appals.get_physical_plan(shownops=None)

# fallbacks in query

In [ ]:
p=appals.get_physical_plan()